In [1]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format

# QUERY SETTLEMENT

In [2]:
settled = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\2024\01. ENERO\SETTLEMENT*.csv', recursive=False)
settled

['C:\\Users\\Cristian.Aparicio\\Documents\\DISPERSIONES\\2024\\01. ENERO\\SETTLEMENT 2024-01-31.csv']

In [3]:
settled_trad = pd.concat([pd.read_csv(f).assign(file_name=os.path.basename(f)) for f in settled])
settled_trad.columns = settled_trad.columns.str.strip().str.lower() 
settled_trad['report_date_local'] = pd.to_datetime(settled_trad['report_date_local'])
settled_trad.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,total_transactions,total_fees_charged,total_taxes,applied_fees_benefits,applied_taxes_benefits,...,total_released,total_retained,tipopago,total_due,type,created_date,created_time,updated_date,updated_time,file_name


In [4]:
settled_trad.shape

(0, 26)

In [5]:
settled_trad['report_date_local'].unique()

<DatetimeArray>
[]
Length: 0, dtype: datetime64[ns]

In [6]:
settled_trad['type'].unique()

array([], dtype=object)

In [7]:
duplicates_data = settled_trad[settled_trad.duplicated(['report_id'] , keep=False)]
duplicates_data.sort_values(by=['tipopago'],ascending=True,inplace=True)
duplicates_data.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,total_transactions,total_fees_charged,total_taxes,applied_fees_benefits,applied_taxes_benefits,...,total_released,total_retained,tipopago,total_due,type,created_date,created_time,updated_date,updated_time,file_name


In [8]:
duplicates_data.shape

(0, 26)

# VALIDACIÓN SETTLEMENT

In [9]:
settled_trad['total_due_calculado'] = (settled_trad['total_transactions']
                                                - settled_trad['total_fees_charged']
                                                - settled_trad['total_taxes']
                                                + settled_trad['applied_fees_benefits']
                                                + settled_trad['applied_taxes_benefits']
                                                + settled_trad['fee_direct_deposit']
                                                + settled_trad['taxes_direct_deposit']
                                                - (-settled_trad['total_adjustments'])
                                                + (-settled_trad['total_adjustment_fees_charged'])
                                                + (-settled_trad['total_adjustment_taxes'])
                                                - settled_trad['cobranza']
                                                + settled_trad['total_released']
                                                - settled_trad['total_retained'])
settled_trad['diferencias'] = settled_trad['total_due_calculado'] - settled_trad['total_due']
settled_trad.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,total_transactions,total_fees_charged,total_taxes,applied_fees_benefits,applied_taxes_benefits,...,tipopago,total_due,type,created_date,created_time,updated_date,updated_time,file_name,total_due_calculado,diferencias


In [10]:
settled_trad.shape

(0, 28)

In [11]:
validation = settled_trad.groupby(['report_date_local','type']).agg({'total_transactions':sum,'total_fees_charged':sum
                                                                    ,'total_taxes':sum,'applied_fees_benefits':sum
                                                                    ,'applied_taxes_benefits':sum,'fee_direct_deposit':sum
                                                                    ,'taxes_direct_deposit':sum,'total_adjustments':sum     
                                                                    ,'total_adjustment_fees_charged':sum
                                                                    ,'total_adjustment_taxes':sum 
                                                                    ,'cobranza':sum,'total_released':sum
                                                                    ,'total_retained':sum,'total_due_calculado':sum
                                                                    ,'total_due':sum
                                                                    ,'diferencias':sum}).reset_index()
validation['total_adjustments'] = validation['total_adjustments']*-1 
validation['total_adjustment_fees_charged'] = validation['total_adjustment_fees_charged']*-1
validation['total_adjustment_taxes'] = validation['total_adjustment_taxes']*-1 
validation.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_12312\3401193542.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  validation = settled_trad.groupby(['report_date_local','type']).agg({'total_transactions':sum,'total_fees_charged':sum
C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_12312\3401193542.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  validation = settled_trad.groupby(['report_date_local','type']).agg({'total_transactions':sum,'total_fees_charged':sum


,report_date_local,type,total_transactions,total_fees_charged,total_taxes,applied_fees_benefits,applied_taxes_benefits,fee_direct_deposit,taxes_direct_deposit,total_adjustments,total_adjustment_fees_charged,total_adjustment_taxes,cobranza,total_released,total_retained,total_due_calculado,total_due,diferencias


In [12]:
validation.shape

(0, 18)

In [13]:
validation['type'].unique()

array([], dtype=object)

In [14]:
path_file = r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\2024\01. ENERO\Dispersión-20240131.xlsx'
book = load_workbook(path_file)
writer = pd.ExcelWriter(path_file, engine='openpyxl', mode='a', if_sheet_exists='overlay')
writer.workbook = openpyxl.load_workbook(path_file)
settled_trad.to_excel(writer, sheet_name='Settlement_All', index=False)
validation.to_excel(writer, sheet_name='Validacion_Settlement', index=False)
book.save(path_file)
writer.close()